# Tabular Q-Learning Training Notebook
This notebook trains and evaluates the tabular Q-learning model.

In [ ]:
import os, sys
from pathlib import Path
PROJECT_ROOT = Path(os.getcwd()).resolve().parent
sys.path.append(str(PROJECT_ROOT))
from src.rl.tabular_q_agent import load_tabular_q_config, train_tabular_q_agent, TabularOfflineDataset
print('PROJECT_ROOT:', PROJECT_ROOT)